# Train Telecom Customer Churn Prediction with XGBoost

This tutorial is based on [this](https://www.kaggle.com/pavanraj159/telecom-customer-churn-prediction/comments#6.-Model-Performances) Kaggle notebook and [this](https://github.com/gojek/feast/tree/master/examples/feast-xgboost-churn-prediction-tutorial) Feast notebook

In [1]:
from hops import featurestore

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
11,application_1601373987661_0017,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
from maggy import Searchspace

# The searchspace can be instantiated with parameters
sp = Searchspace(max_depth=('INTEGER', [2, 50]), learning_rate=('DISCRETE', [0.9, 0.5, 0.2, 0.1, 0.01, 0.001]), gamma=('DOUBLE', [0, 5]), reg_lambda=('DOUBLE', [0, 5]))

Hyperparameter added: max_depth
Hyperparameter added: learning_rate
Hyperparameter added: gamma
Hyperparameter added: reg_lambda

Define training logic in wrapper function:

In [3]:
def train(max_depth, learning_rate, gamma, reg_lambda):
    import pandas as pd
    from hops import hdfs, featurestore
    from hops import pandas_helper as pandas
    from hops import model as hopsmodel
    import pickle
    import warnings
    warnings.filterwarnings("ignore")
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, classification_report
    from xgboost import XGBClassifier
    from torch.utils.tensorboard import SummaryWriter
    from maggy import tensorboard

    # Get path to training dataset
    file_path = featurestore.get_training_dataset_path("telco_churn")
    telecom_df = pandas.read_csv(hdfs.get_plain_path([path for path in hdfs.ls(file_path) if ".csv" in path][0]))
    Id_col     = ['customer_id']
    target_col = ["churn"]
    
    # Split into a train and test set
    train, test = train_test_split(telecom_df,test_size = .25 ,random_state = 111)

    # Seperating dependent and independent variables
    cols    = [i for i in telecom_df.columns if i not in Id_col + target_col]
    training_x = train[cols]
    training_y = train[target_col]
    testing_x  = test[cols]
    testing_y  = test[target_col]    

    # Instantiate classifier with hyperparameters as variables
    model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                          colsample_bytree=1, gamma=gamma, learning_rate=learning_rate, max_delta_step=0,
                          max_depth=max_depth, min_child_weight=1, missing=None, n_estimators=100,
                          n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
                          reg_alpha=0, reg_lambda=reg_lambda, scale_pos_weight=1, seed=None,
                          silent=True, subsample=1)

    # Train model
    model.fit(training_x, training_y)
    predictions = model.predict(testing_x)
    probabilities = model.predict_proba(testing_x)    
    
    coefficients = pd.DataFrame(model.feature_importances_)
    column_df = pd.DataFrame(cols)
    coef_sumry = (pd.merge(coefficients, column_df, left_index=True,
                           right_index=True, how="left"))
    coef_sumry.columns = ["coefficients", "features"]
    coef_sumry = coef_sumry.sort_values(by="coefficients", ascending=False)

    accuracy=accuracy_score(testing_y, predictions)

    print("\n Classification report : \n", classification_report(testing_y, predictions))
    print("Accuracy   Score : ", accuracy)

    # use any tensorboard writer
    writer = SummaryWriter(tensorboard.logdir()+"/validation")
    writer.add_scalar('epoch_acc', accuracy, 1)
    writer.close()
    
    
    MODEL_NAME = "XGBoost_Churn_Classifier"
    file_name = "xgb_reg.pkl"
    hdfs_path = "Resources/xgboost_modeller"

    pickle.dump(model, open(file_name, "wb"))
    hdfs.mkdir(hdfs_path)
    hdfs.copy_to_hdfs(file_name, hdfs_path, overwrite=True)

    # test that we can load and use the model
    xgb_model_loaded = pickle.load(open(file_name, "rb"))
    xgb_model_loaded.predict(testing_x)[0] == model.predict(testing_x)[0]

    # save to the model registry
    hopsmodel.export(hdfs_path, MODEL_NAME, metrics={'accuracy': accuracy})    
    
    
    return accuracy

In [4]:
from maggy import experiment
result = experiment.lagom(train, 
                           searchspace=sp, 
                           optimizer='randomsearch', 
                           direction='max',
                           num_trials=10, 
                           name='CHURN',
                           hb_interval=5, 
                           es_interval=5,
                           es_min=5
                          )

Started Maggy Experiment: CHURN, application_1601373987661_0017, run 1

------ RandomSearch Results ------ direction(max) 
BEST combination {"max_depth": 11, "learning_rate": 0.01, "gamma": 2.9992478857741984, "reg_lambda": 1.7847427765057688} -- metric 0.8141414141414142
WORST combination {"max_depth": 47, "learning_rate": 0.9, "gamma": 1.779845514714643, "reg_lambda": 0.6473636401114952} -- metric 0.7686868686868686
AVERAGE metric -- 0.7975757575757575
EARLY STOPPED Trials -- 0
Total job time 0 hours, 1 minutes, 26 seconds

Finished ExperimentWARN: Can't reach Maggy server. No progress information and logs available. Job continues running anyway.
